In [30]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip
from core.s3 import S3AssetManager

In [31]:
from typing import Iterable, Mapping, Tuple, Dict, Any
import plotly.express as px
import plotly.graph_objs as go

import seaborn as sns
import re
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from itertools import product
import warnings
warnings.filterwarnings("ignore")


In [32]:
notebook_name = "comayma_segundo_ensayo_inhimol"
s3 = S3AssetManager(notebook_name=notebook_name)

In [33]:
def to_log10_plus1(x: np.ndarray) -> np.ndarray:
    """Transformación log10(UFC + 1). Clipa en 0 para evitar negativos por error."""
    x = np.asarray(x, dtype=float)
    return np.log10(np.clip(x, a_min=0, a_max=None) + 1.0)

In [34]:


df_raw = s3.read_excel("raw/comayma/ensayos/resultados_ensayo3_oct_comayma.xlsx", sheet_name="M.B. 2da Prueba")
df = pd.melt(
    df_raw,
    id_vars=["Lugar", "Etapa","Fecha muestreo", "Dosis", "Tratamiento", "Muestra"],
    value_vars=["MyL", "Coliformes", "E. coli", "Aerobios mesófilos"],
    var_name="Microorganismo",
    value_name="Resultado"
)

df['concat'] = df['Etapa'].astype(str) + ' ' + df['Fecha muestreo'].astype(str)
df['Resultado'] = pd.to_numeric(df['Resultado'], errors='coerce').fillna(1)
df['Resultado'] = df['Resultado'].replace(0, 1)

df['Resultado_log10'] = np.where(df['Resultado'] > 0, np.log10(df['Resultado']), 0)

df["Resultado"] = pd.to_numeric(df["Resultado"], errors="coerce").fillna(0)
df["resultado_clipped"] = df["Resultado"].clip(lower=1.0)  # 1 UFC como mínimo medible
df["log10_Resultado"] = np.log10(df["resultado_clipped"])
df = df[df["Microorganismo"].isin(['MyL', 'Coliformes', 'Aerobios mesófilos'])]

ValueError: Worksheet named 'M.B. 2da Prueba' not found

In [19]:
df

,Lugar,Etapa,Fecha muestreo,Dosis,Tratamiento,Muestra,Microorganismo,Resultado,concat,Resultado_log10,resultado_clipped,log10_Resultado
0,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 1,MyL,160.0,10 min aplicado 2025-08-16,2.204120,160.0,2.204120
1,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 2,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
2,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 3,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
3,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 4,MyL,80.0,10 min aplicado 2025-08-16,1.903090,80.0,1.903090
4,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 5,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
131,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 2,Aerobios mesófilos,1900.0,Seguimiento 2025-08-26,3.278754,1900.0,3.278754
132,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 3,Aerobios mesófilos,3000.0,Seguimiento 2025-08-26,3.477121,3000.0,3.477121
133,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 4,Aerobios mesófilos,1000.0,Seguimiento 2025-08-26,3.000000,1000.0,3.000000
134,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 5,Aerobios mesófilos,790.0,Seguimiento 2025-08-26,2.897627,790.0,2.897627


In [20]:


# ==============
resumen_por_etapa = (
    df.groupby(['Tratamiento', 'Microorganismo', 'Etapa'])['Resultado']
      .agg(['mean', 'std', 'count'])
      .reset_index()
)
resumen_por_etapa

,Tratamiento,Microorganismo,Etapa,mean,std,count
0,Inhimold 50 CF,Aerobios mesófilos,10 min aplicado,448.571429,244.705849,7
1,Inhimold 50 CF,Aerobios mesófilos,Recepción mp,2195.000000,3083.304504,4
2,Inhimold 50 CF,Aerobios mesófilos,Seguimiento,223.333333,69.474216,6
3,Inhimold 50 CF,Coliformes,10 min aplicado,8.000000,18.520259,7
4,Inhimold 50 CF,Coliformes,Recepción mp,35.250000,32.816408,4
5,Inhimold 50 CF,Coliformes,Seguimiento,1.000000,0.000000,6
6,Inhimold 50 CF,MyL,10 min aplicado,156.142857,198.762361,7
7,Inhimold 50 CF,MyL,Recepción mp,1687.500000,1130.586131,4
8,Inhimold 50 CF,MyL,Seguimiento,291.666667,128.750405,6
9,Producto competidor,Aerobios mesófilos,10 min aplicado,3588.571429,6398.795274,7


In [21]:
# Pivot ancho para reducciones (vía medias)
mat = resumen_por_etapa.pivot_table(
    index=['Tratamiento', 'Microorganismo'],
    columns='Etapa',
    values=['mean', 'std', 'count']
)
mat.columns = ['_'.join(col) for col in mat.columns.values]
mat

count_10 min aplicado  \
Tratamiento         Microorganismo                              
Inhimold 50 CF      Aerobios mesófilos                      7   
                    Coliformes                              7   
                    MyL                                     7   
Producto competidor Aerobios mesófilos                      7   
                    Coliformes                              7   
                    MyL                                     7   

                                        count_Recepción mp  count_Seguimiento  \
Tratamiento         Microorganismo                                              
Inhimold 50 CF      Aerobios mesófilos                   4                  6   
                    Coliformes                           4                  6   
                    MyL                                  4                  6   
Producto competidor Aerobios mesófilos                   4                  6   
                    Coliformes                           4                  6   
                    MyL                                  4                  6   

                                        mean_10 min aplicado  \
Tratamiento         Microorganismo                             
Inhimold 50 CF      Aerobios mesófilos            448.571429   
                    Coliformes                      8.000000   
                    MyL                           156.142857   
Producto competidor Aerobios mesófilos           3588.571429   
                    Coliformes                     53.142857   
                    MyL                          1022.857143   

                                        mean_Recepción mp  mean_Seguimiento  \
Tratamiento         Microorganismo                                            
Inhimold 50 CF      Aerobios mesófilos            2195.00        223.333333   
                    Coliformes                      35.25          1.000000   
                    MyL                           1687.50        291.666667   
Producto competidor Aerobios mesófilos            1975.00       1423.333333   
                    Coliformes                       8.00         10.666667   
                    MyL                           1500.00        468.333333   

                                        std_10 min aplicado  std_Recepción mp  \
Tratamiento         Microorganismo                                              
Inhimold 50 CF      Aerobios mesófilos           244.705849       3083.304504   
                    Coliformes                    18.520259         32.816408   
                    MyL                          198.762361       1130.586131   
Producto competidor Aerobios mesófilos          6398.795274       1071.991915   
                    Coliformes                   100.939633          9.055385   
                    MyL                         1154.523196       1017.054571   

                                        std_Seguimiento  
Tratamiento         Microorganismo                       
Inhimold 50 CF      Aerobios mesófilos        69.474216  
                    Coliformes                 0.000000  
                    MyL                      128.750405  
Producto competidor Aerobios mesófilos       887.843830  
                    Coliformes                19.602721  
                    MyL                      161.420775

In [22]:
# Reducciones log con medias (interpretación estándar en micro: log-reduction)
def safe_log10(x):
    x = np.asarray(x, dtype=float)
    x = np.clip(x, 0, None)  # seguridad
    return np.log10(x)

In [23]:
for etapa in ["Recepción mp","10 min aplicado","Seguimiento"]:
    if f"mean_{etapa}" not in mat.columns:
        # si falta alguna etapa, crear NaN
        mat[f"mean_{etapa}"] = np.nan

mat["Reduccion_Log_10_min"] = safe_log10(mat["mean_Recepción mp"]) - safe_log10(mat["mean_10 min aplicado"])
mat["Reduccion_Log_Seguimiento"] = safe_log10(mat["mean_Recepción mp"]) - safe_log10(mat["mean_Seguimiento"])
mat = mat.reset_index()
mat

,Tratamiento,Microorganismo,count_10 min aplicado,count_Recepción mp,count_Seguimiento,mean_10 min aplicado,mean_Recepción mp,mean_Seguimiento,std_10 min aplicado,std_Recepción mp,std_Seguimiento,Reduccion_Log_10_min,Reduccion_Log_Seguimiento
0,Inhimold 50 CF,Aerobios mesófilos,7,4,6,448.571429,2195.00,223.333333,244.705849,3083.304504,69.474216,0.689603,0.992481
1,Inhimold 50 CF,Coliformes,7,4,6,8.000000,35.25,1.000000,18.520259,32.816408,0.000000,0.644069,1.547159
2,Inhimold 50 CF,MyL,7,4,6,156.142857,1687.50,291.666667,198.762361,1130.586131,128.750405,1.033722,0.762357
3,Producto competidor,Aerobios mesófilos,7,4,6,3588.571429,1975.00,1423.333333,6398.795274,1071.991915,887.843830,-0.259354,0.142260
4,Producto competidor,Coliformes,7,4,6,53.142857,8.00,10.666667,100.939633,9.055385,19.602721,-0.822355,-0.124939
5,Producto competidor,MyL,7,4,6,1022.857143,1500.00,468.333333,1154.523196,1017.054571,161.420775,0.166276,0.505536


In [24]:


def plot_reduccion_tratamientos_plotly(
    df: pd.DataFrame,
    metric: str = "10min",            # "10min" o "seguimiento"
    prefer: str = "Inhimold 50 CF",   # tratamiento para ordenar por impacto
    color_map: dict | None = None,    # {"Inhimold 50 CF":"#1C8074", "Producto competidor":"#94AF92"}
    output_dir: str | None = None,    # p.ej., ROOT_IMAGEN
    filename: str | None = None,
    save_html: bool = True,
    save_png: bool = False,           # requiere kaleido para exportar PNG
    width: int = 1200,
    height: int = 620,
):
    """
    Devuelve: fig (plotly.Figure), bar_df (DataFrame usado), paths (dict con rutas guardadas si aplica).
    """
    assert metric in {"10min", "seguimiento"}, "metric debe ser '10min' o 'seguimiento'"

    def safe_log10(x):
        x = np.asarray(x, dtype=float)
        x = np.clip(x, 1e-9, None)
        return np.log10(x)

    # --- Resumen por etapa ---
    resumen = (
        df.groupby(["Tratamiento", "Microorganismo", "Etapa"])["Resultado"]
          .agg(["mean", "std", "count"])
          .reset_index()
    )

    mat = resumen.pivot_table(
        index=["Tratamiento", "Microorganismo"],
        columns="Etapa",
        values=["mean", "std", "count"]
    )
    mat.columns = ["_".join(col) for col in mat.columns.values]
    mat = mat.reset_index()

    # --- Reducciones (ambas) ---
    # Recepción → 10 min
    mat["Reduccion_Log_10_min"] = (
        safe_log10(mat.get("mean_Recepción mp", np.nan)) -
        safe_log10(mat.get("mean_10 min aplicado", np.nan))
    )
    # Recepción → Seguimiento
    mat["Reduccion_Log_Seguimiento"] = (
        safe_log10(mat.get("mean_Recepción mp", np.nan)) -
        safe_log10(mat.get("mean_Seguimiento", np.nan))
    )

    metric_col = "Reduccion_Log_10_min" if metric == "10min" else "Reduccion_Log_Seguimiento"
    y_title = f"Reducción log10 (Recepción → {'10 min' if metric=='10min' else 'Seguimiento'})"
    titulo = "Reducción inmediata" if metric == "10min" else "Reducción residual"
    title = f"{titulo} en log10 por microorganismo — Comparativo de tratamientos"

    bar_df = mat[["Tratamiento", "Microorganismo", metric_col]].dropna().copy()
    bar_df.rename(columns={metric_col: "Reduccion"}, inplace=True)

    # --- Orden profesional por impacto del tratamiento preferido ---
    if prefer in bar_df["Tratamiento"].unique():
        order_series = (
            bar_df[bar_df["Tratamiento"] == prefer]
            .set_index("Microorganismo")["Reduccion"]
            .sort_values(ascending=False)
        )
        micro_order = list(order_series.index)
    else:
        micro_order = sorted(bar_df["Microorganismo"].unique())

    # Orden de tratamientos (primero prefer y competidor si existe)
    trat_order = [prefer]
    if "Producto competidor" in bar_df["Tratamiento"].unique() and "Producto competidor" != prefer:
        trat_order.append("Producto competidor")
    # agrega cualquier otro tratamiento que exista
    trat_order += [t for t in bar_df["Tratamiento"].unique() if t not in trat_order]

    # --- Colores corporativos por defecto ---
    if color_map is None:
        color_map = {
            "Inhimold 50 CF": "#1C8074",
            "Producto competidor": "#666666",
        }

    # --- Construcción de figura ---
    fig = go.Figure()
    for tr in trat_order:
        sub = bar_df[bar_df["Tratamiento"] == tr].set_index("Microorganismo").reindex(micro_order)
        y_vals = sub["Reduccion"].values
        # % de reducción equivalente: 1 - 10^(-L)
        perc_red = (1 - np.power(10.0, -np.clip(y_vals, a_min=-10, a_max=10))) * 100.0

        fig.add_trace(go.Bar(
            x=micro_order,
            y=y_vals,
            name=tr,
            marker=dict(color=color_map.get(tr, "#4c78a8"), line=dict(width=0)),  # sin bordes
            text=[f"{v:.2f}" if np.isfinite(v) else "" for v in y_vals],
            textposition="outside",
            cliponaxis=False,
            customdata=np.stack([y_vals, perc_red], axis=-1),
            hovertemplate=(
                "<b>%{x}</b><br>"
                f"Tratamiento: <b>{tr}</b><br>"
                "Reducción log10: %{customdata[0]:.3f}<br>"
                "Reducción equivalente: %{customdata[1]:.1f}%<extra></extra>"
            )
        ))

    # --- Estilo profesional ---
    fig.update_layout(
        title={"text": title, "x": 0.02, "xanchor": "left"},
        paper_bgcolor="white",
        plot_bgcolor="white",
        font=dict(family="Arial", size=15, color="black"),
        barmode="group",
        bargroupgap=0.12,
        bargap=0.18,
        legend=dict(
            title="Tratamiento",
            orientation="h",
            yanchor="bottom", y=1.02,
            xanchor="left", x=0.02,
            bgcolor="rgba(255,255,255,0)",
            borderwidth=0
        ),
        margin=dict(l=70, r=40, t=80, b=90),
        height=height,
        width=width
    )
    fig.update_xaxes(
        title="Microorganismo",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.update_yaxes(
        title=y_title,
        zeroline=False,
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.add_hline(y=0, line_width=2, line_color="black")
    fig.show()

    # --- Guardado opcional ---
    paths = {}
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        if filename is None:
            filename = f"barras_reduccion_{'10min' if metric=='10min' else 'seguimiento'}"
        html_path = os.path.join(output_dir, f"{filename}.html")
        if save_html:
            fig.write_html(html_path, include_plotlyjs="cdn", full_html=True)
            paths["html"] = html_path
        if save_png:
            try:
                png_path = os.path.join(output_dir, f"{filename}.png")
                fig.write_image(png_path, scale=3, width=width, height=height)  # requiere kaleido
                paths["png"] = png_path
            except Exception:
                pass

    return fig, bar_df, paths


In [25]:
df

,Lugar,Etapa,Fecha muestreo,Dosis,Tratamiento,Muestra,Microorganismo,Resultado,concat,Resultado_log10,resultado_clipped,log10_Resultado
0,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 1,MyL,160.0,10 min aplicado 2025-08-16,2.204120,160.0,2.204120
1,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 2,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
2,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 3,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
3,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 4,MyL,80.0,10 min aplicado 2025-08-16,1.903090,80.0,1.903090
4,SILO 9 INHIMOLD,10 min aplicado,2025-08-16,1.5,Inhimold 50 CF,Muestra 5,MyL,1.0,10 min aplicado 2025-08-16,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
131,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 2,Aerobios mesófilos,1900.0,Seguimiento 2025-08-26,3.278754,1900.0,3.278754
132,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 3,Aerobios mesófilos,3000.0,Seguimiento 2025-08-26,3.477121,3000.0,3.477121
133,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 4,Aerobios mesófilos,1000.0,Seguimiento 2025-08-26,3.000000,1000.0,3.000000
134,SILO 6 MYCOCURB,Seguimiento,2025-08-26,0.5,Producto competidor,Muestra 5,Aerobios mesófilos,790.0,Seguimiento 2025-08-26,2.897627,790.0,2.897627


In [26]:
# 1) Reducción inmediata (Recepción → 10 min)
fig1, data1, paths1 = plot_reduccion_tratamientos_plotly(
    df,
    metric="10min",
    save_html=True,
    save_png=False
)

s3.save_plotly_html(fig1, "barras_reduccion_10min.html")

In [27]:
# 2) Reducción residual (Recepción → Seguimiento)
fig2, data2, paths2 = plot_reduccion_tratamientos_plotly(
    df,
    metric="seguimiento",
    save_html=True,
    save_png=False
)
s3.save_plotly_html(fig2, "barras_reduccion_seguimiento.html")

In [28]:


COLOR_MAP = {
    "Inhimold 50 CF": "#1C8074",
    "Producto competidor": "#666666",
}
ETAPAS_ORDER = ["Recepción mp", "10 min aplicado", "Seguimiento"]
ETIQUETAS = {"Recepción mp": "Recepción", "10 min aplicado": "10 min", "Seguimiento": "Seguimiento"}

def plot_half_violins_dual_trat_fixed_layout(
    df: pd.DataFrame,
    tratamiento_left: str,
    tratamiento_right: str,
    filename: str | None = None
) -> str:
    sub = df[df["Tratamiento"].isin([tratamiento_left, tratamiento_right])].copy()
    sub = sub[sub["Etapa"].isin(ETAPAS_ORDER)]
    sub["Etapa_label"] = sub["Etapa"].map(ETIQUETAS)
    sub["Etapa_label"] = pd.Categorical(sub["Etapa_label"], categories=[ETIQUETAS[e] for e in ETAPAS_ORDER], ordered=True)

    fig = go.Figure()
    first_left = True
    first_right = True

    for etapa_raw in ETAPAS_ORDER:
        etapa_lbl = ETIQUETAS[etapa_raw]

        # Left (negative side)
        vals_left = sub.loc[(sub["Etapa"] == etapa_raw) & (sub["Tratamiento"] == tratamiento_left), "log10_Resultado"].astype(float)
        if not vals_left.empty:
            fig.add_trace(go.Violin(
                x=[etapa_lbl]*len(vals_left),
                y=vals_left,
                name=tratamiento_left,
                legendgroup=tratamiento_left,
                showlegend=first_left,
                side="negative",
                width=0.9,
                points=False,
                scalemode="width",
                meanline_visible=False,
                box_visible=False,
                line=dict(width=1.2, color=COLOR_MAP.get(tratamiento_left, "#1C8074")),
                fillcolor=COLOR_MAP.get(tratamiento_left, "#1C8074"),
                opacity=0.8
            ))
            first_left = False

        # Right (positive side)
        vals_right = sub.loc[(sub["Etapa"] == etapa_raw) & (sub["Tratamiento"] == tratamiento_right), "log10_Resultado"].astype(float)
        if not vals_right.empty:
            fig.add_trace(go.Violin(
                x=[etapa_lbl]*len(vals_right),
                y=vals_right,
                name=tratamiento_right,
                legendgroup=tratamiento_right,
                showlegend=first_right,
                side="positive",
                width=0.9,
                points=False,
                scalemode="width",
                meanline_visible=False,
                box_visible=False,
                line=dict(width=1.2, color=COLOR_MAP.get(tratamiento_right, "#94AF92")),
                fillcolor=COLOR_MAP.get(tratamiento_right, "#94AF92"),
                opacity=0.8
            ))
            first_right = False

    # Medianas por etapa como puntos
    for tr_name, color in [(tratamiento_left, COLOR_MAP.get(tratamiento_left, "#1C8074")),
                           (tratamiento_right, COLOR_MAP.get(tratamiento_right, "#94AF92"))]:
        med = (
            sub[sub["Tratamiento"] == tr_name]
            .groupby("Etapa_label", observed=True)["log10_Resultado"]
            .mean()
            .reindex([ETIQUETAS[e] for e in ETAPAS_ORDER])
        )
        fig.add_trace(go.Scatter(
            x=med.index.tolist(),
            y=med.values.astype(float).tolist(),
            mode="markers",
            name=f"Promedio — {tr_name}",
            legendgroup=tr_name,
            marker=dict(size=9, color=color, line=dict(width=0))
        ))

    # ---- Layout: título multilínea + leyenda abajo + márgenes amplios ----
    title_text = (
        "Distribución de las observaciones por etapa<br>"
        f"<sup>{tratamiento_left} (izq) vs {tratamiento_right} (der)</sup>"
    )

    fig.update_layout(
        title=dict(
            text=title_text,
            x=0.02, xanchor="left",
            y=0.92, yanchor="top",
            font=dict(size=20),
            pad=dict(t=6, b=6, l=0, r=0)
        ),
        paper_bgcolor="white", plot_bgcolor="white",
        font=dict(family="Arial", size=15, color="black"),
        margin=dict(l=70, r=40, t=110, b=110),   # más espacio arriba/abajo
        height=500, width=1100,
        legend=dict(
            orientation="h",
            yanchor="top", y=-0.12,   # leyenda debajo del gráfico
            xanchor="left", x=0.02,
            bgcolor="rgba(255,255,255,0)",
            borderwidth=0
        )
    )
    fig.update_xaxes(
        title="Etapa",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        categoryorder="array", categoryarray=[ETIQUETAS[e] for e in ETAPAS_ORDER],
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.update_yaxes(
        title="log10(UFC)",
        showline=True, linewidth=2, linecolor="black",
        ticks="outside", tickcolor="black", tickwidth=1.5,
        gridcolor="rgba(0,0,0,0.08)"
    )
    fig.add_annotation(
    x=0.02, y=1.12, xref="paper", yref="paper",
    text="Entre Recepción y Seguimiento: Inhimold 50 CF ≈ 29.5 × menos (−96%); competidor ≈ 4.46 × menos (−77%)",
    showarrow=False, align="left",
    font=dict(size=20, color="black"))
    fig.show()
    return fig

# Ejecutar con el par del dataset
treats = list(df["Tratamiento"].dropna().unique())
new_path = None
f = plot_half_violins_dual_trat_fixed_layout(df, treats[0], treats[1])

s3.save_plotly_html(f, "half_violins_fixed_Inhimold_50_CF_vs_Producto_competidor.html")

